In [1]:
import pycuda.driver as driver
import pycuda.autoinit

In [2]:
kernel = """
    __global__ void matrix_prod(float *a, float *b, float *c, int n, int l, int m)
    {
        // Identificación del hilo 2D en un bloque
        int i = blockIdx.x * blockDim.x + threadIdx.y; // Fila i
        int j = blockIdx.y * blockDim.y + threadIdx.x; // Columna j

        // Verificamos estar dentro de los límites (n x m)
        if(i < n && j < m) {
            float valor = 0;
            for (int k = 0; k < l; k++) {
                valor += a[i * l  + k] * b[k * m + j];
            }
        // Almacenamos el resultado
        c[i * m + j] = valor;
        }
    }
"""

In [3]:
import numpy as np
import time

# Definimos el tamaño de las matrices A(n x l) y B (l x m)
m = np.int32(4000)
l = np.int32(6000)
n = np.int32(2000)

# creamos dos matrices aleatorias
np.random.seed(13)
a_cpu = np.random.randn(m, l).astype(np.float32)
b_cpu = np.random.randn(l, n).astype(np.float32)

# calculamos el resultado en la CPU para comparar con el de la GPU

start_time_CPU = time.time()
c_cpu = np.dot(a_cpu, b_cpu)
end_time_CPU = time.time()
elapsed_time_CPU = end_time_CPU - start_time_CPU

print(f"El tiempo de ejecución fue: {elapsed_time_CPU} segundos")

El tiempo de ejecución fue: 0.31313204765319824 segundos


In [4]:
from math import sqrt, ceil

device = driver.Device(0)
max_threads_per_block = device.get_attribute(driver.device_attribute.MAX_THREADS_PER_BLOCK)
t_xy = int(ceil(sqrt(max_threads_per_block)))
NBlocks_x = ceil(m / t_xy)
NBlocks_y = ceil(n / t_xy)
print(f"Bloques en x: {NBlocks_x}, bloques en y: {NBlocks_y}, con t_xy: {t_xy}")

Bloques en x: 125, bloques en y: 63, con t_xy: 32


In [5]:
from pycuda import compiler

# Compilamos el kernel y obtenemos el handler a la función
mod = compiler.SourceModule(kernel)
func = mod.get_function("matrix_prod")

In [6]:
from pycuda import gpuarray, tools

# transferimos de la memoria host (CPU) a la memoria del device (GPU)
start_time_CPU2GPU = time.time()
a_gpu = gpuarray.to_gpu(a_cpu) 
b_gpu = gpuarray.to_gpu(b_cpu)
end_time_CPU2GPU = time.time()
elapsed_time_CPU2GPU = end_time_CPU2GPU - start_time_CPU2GPU
# creamos un array vacío en la GPU para el resultado (C = A . B)
c_gpu = gpuarray.empty((m, n), np.float32)

print(f"El tiempo de transferencia de datos fue: {elapsed_time_CPU2GPU} segundos")

# Ejecutamos el kernel en la GPU
start_time_GPU = time.time()
func(a_gpu, b_gpu, c_gpu, m, l, n, block=(t_xy, t_xy, 1), grid=(NBlocks_x, NBlocks_y, 1))
end_time_GPU = time.time()
elapsed_time_GPU = end_time_GPU - start_time_GPU

print(f"El tiempo de ejecución fue: {elapsed_time_GPU} segundos")

El tiempo de transferencia de datos fue: 0.03423953056335449 segundos
El tiempo de ejecución fue: 0.0003933906555175781 segundos


In [9]:
np.allclose(c_cpu, c_gpu.get(), rtol=1e-05, atol=1e-03)

True


### Copyright 2020-2025 Facundo Batista y Manuel Carlevaro

Licencia CC BY-NC-SA 4.0

Para más info visitar: https://github.com/facundobatista/libro-pyciencia/

